In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["LANGSMITH_TRACING"] = "true"

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("What is the weather in Caracas, Venezuela")
print(search_results)
tools = [search]

[{'title': 'Weather in Caracas, Venezuela', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Caracas', 'region': 'Distrito Federal', 'country': 'Venezuela', 'lat': 10.5, 'lon': -66.9167, 'tz_id': 'America/Caracas', 'localtime_epoch': 1743639368, 'localtime': '2025-04-02 20:16'}, 'current': {'last_updated_epoch': 1743639300, 'last_updated': '2025-04-02 20:15', 'temp_c': 19.4, 'temp_f': 66.9, 'is_day': 0, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png', 'code': 1030}, 'wind_mph': 4.3, 'wind_kph': 6.8, 'wind_degree': 81, 'wind_dir': 'E', 'pressure_mb': 1015.0, 'pressure_in': 29.97, 'precip_mm': 0.03, 'precip_in': 0.0, 'humidity': 95, 'cloud': 71, 'feelslike_c': 19.4, 'feelslike_f': 66.9, 'windchill_c': 19.4, 'windchill_f': 66.9, 'heatindex_c': 19.4, 'heatindex_f': 66.9, 'dewpoint_c': 18.7, 'dewpoint_f': 65.6, 'vis_km': 2.0, 'vis_miles': 1.0, 'uv': 0.0, 'gust_mph': 6.6, 'gust_kph': 10.7}}", 'score': 0.9081421}, {'title': '10-

In [7]:
from langchain.chat_models import init_chat_model

model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")


In [8]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="How is the weather usually in Canada?")])
print(response.content)

Canada is a vast and geographically diverse country, spanning across six time zones and covering a wide range of climates and weather conditions. The weather in Canada can vary greatly depending on the region, season, and elevation. Here's a general overview of the typical weather patterns in different parts of Canada:

** Seasons:**

1. **Winter (December to February):** Cold temperatures, snow, and ice are common in most parts of Canada, especially in the provinces of Quebec, Ontario, and the Prairies. Temperatures can drop as low as -20°C to -30°C (-4°F to -22°F) in some areas.
2. **Spring (March to May):** Mild temperatures, with occasional rain and snow showers, characterize this season. Daytime temperatures can rise to 10°C to 20°C (50°F to 68°F), while nighttime temperatures can still be cool.
3. **Summer (June to August):** Warm to hot temperatures, with average highs ranging from 20°C to 30°C (68°F to 86°F), are typical in most parts of Canada. Humidity can be high in some reg

In [9]:
model_with_tools = model.bind_tools(tools)

In [10]:
response2 = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response2.content}")
print(f"ToolCalls: {response2.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather today'}, 'id': 'call_g6mv', 'type': 'tool_call'}]


In [11]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [12]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools, checkpointer=memory)
config = {"configurable" : { "thread_id" : "abc123"}}

In [13]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hola. Me llamo Carlos, Me pudieras decir que tan seguro es un tesla cybertruck?")]}, 
    config,
    stream_mode="values",
):
    print(chunk)
    chunk["messages"][-1].pretty_print()
    print("----")

{'messages': [HumanMessage(content='hola. Me llamo Carlos, Me pudieras decir que tan seguro es un tesla cybertruck?', additional_kwargs={}, response_metadata={}, id='2abe1461-106f-46f9-8c20-f4073469dd13')]}
================================ Human Message =================================

hola. Me llamo Carlos, Me pudieras decir que tan seguro es un tesla cybertruck?
----
{'messages': [HumanMessage(content='hola. Me llamo Carlos, Me pudieras decir que tan seguro es un tesla cybertruck?', additional_kwargs={}, response_metadata={}, id='2abe1461-106f-46f9-8c20-f4073469dd13'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_k33s', 'function': {'arguments': '{"query": "Tesla Cybertruck safety"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 301, 'total_tokens': 322, 'completion_time': 0.076363636, 'prompt_time': 0.02717114, 'queue_time': 0.19917634399999998, 'total_time': 0.103

In [14]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Me puedes decir como me llamo?")]}, 
    config,
    stream_mode="values",
):
    print(chunk)
    chunk["messages"][-1].pretty_print()
    print("----")

{'messages': [HumanMessage(content='hola. Me llamo Carlos, Me pudieras decir que tan seguro es un tesla cybertruck?', additional_kwargs={}, response_metadata={}, id='2abe1461-106f-46f9-8c20-f4073469dd13'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_k33s', 'function': {'arguments': '{"query": "Tesla Cybertruck safety"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 301, 'total_tokens': 322, 'completion_time': 0.076363636, 'prompt_time': 0.02717114, 'queue_time': 0.19917634399999998, 'total_time': 0.103534776}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_9a8b91ba77', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-85b78597-2c42-4d27-9f93-8598c4687929-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Tesla Cybertruck safety'}, 'id': 'call_k33s', 'type': 'tool_call'}], usage_metadata={'input_tokens': 301, 'output_to